In [1]:
import os
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import sys
#!{sys.executable} -m pip install librosa
import librosa
import librosa.display
import ast
#!{sys.executable} -m pip install utils
import utils
#!{sys.executable} -m pip install squarify
import squarify
#!{sys.executable} -m pip install pandas_profiling
import pandas_profiling
import time

pd.set_option('display.max_columns',None)

%matplotlib inline

In [2]:
def load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    'category', categories=SUBSETS, ordered=True)
        except (ValueError, TypeError):
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks

In [3]:
# Directory where mp3s are stored

AUDIO_DIR = os.environ.get('fma_small')

# Load metadata

tracks = load('C:\\Users\\julia.wlastowska\\Desktop\\inz\\fma_metadata\\tracks.csv')
genres = load('C:\\Users\\julia.wlastowska\\Desktop\\inz\\fma_metadata\\genres.csv')
echonest = load('C:\\Users\\julia.wlastowska\\Desktop\\inz\\fma_metadata\\echonest.csv')
features = load('C:\\Users\\julia.wlastowska\\Desktop\\inz\\fma_metadata\\features.csv')

tracks.shape, genres.shape, echonest.shape, features.shape

((106574, 52), (163, 4), (13129, 249), (106574, 518))

In [4]:
tracks=tracks.drop(columns=[('album', 'comments')])
tracks=tracks.drop(columns=[('album', 'date_released')])
tracks=tracks.drop(columns=[('album', 'date_created')])
tracks=tracks.drop(columns=[('album', 'engineer')])
tracks=tracks.drop(columns=[('album', 'favorites')])
tracks=tracks.drop(columns=[('album', 'id')])
tracks=tracks.drop(columns=[('album', 'information')])
tracks=tracks.drop(columns=[('album', 'producer')])
tracks=tracks.drop(columns=[('album', 'tracks')])
tracks=tracks.drop(columns=[('album', 'type')])
tracks=tracks.drop(columns=[('album', 'tags')])
tracks=tracks.drop(columns=[('artist', 'active_year_begin')])
tracks=tracks.drop(columns=[('artist', 'active_year_end')])
tracks=tracks.drop(columns=[('artist', 'associated_labels')])
tracks=tracks.drop(columns=[('artist', 'bio')])
tracks=tracks.drop(columns=[('artist', 'comments')])
tracks=tracks.drop(columns=[('artist', 'date_created')])
tracks=tracks.drop(columns=[('artist', 'members')])
tracks=tracks.drop(columns=[('artist', 'related_projects')])
tracks=tracks.drop(columns=[('artist', 'tags')])
tracks=tracks.drop(columns=[('artist', 'website')])
tracks=tracks.drop(columns=[('artist', 'wikipedia_page')])
tracks=tracks.drop(columns=[('track', 'comments')])
tracks=tracks.drop(columns=[('track', 'composer')])
tracks=tracks.drop(columns=[('track', 'date_recorded')])
tracks=tracks.drop(columns=[('track', 'information')])
tracks=tracks.drop(columns=[('track', 'license')])
tracks=tracks.drop(columns=[('track', 'lyricist')])
tracks=tracks.drop(columns=[('track', 'number')])
tracks=tracks.drop(columns=[('track', 'tags')])
tracks=tracks.drop(columns=[('track', 'publisher')])

In [5]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106574 entries, 2 to 155320
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   (album, listens)        106574 non-null  int64         
 1   (album, title)          105549 non-null  object        
 2   (artist, favorites)     106574 non-null  int64         
 3   (artist, id)            106574 non-null  int64         
 4   (artist, latitude)      44544 non-null   float64       
 5   (artist, location)      70210 non-null   object        
 6   (artist, longitude)     44544 non-null   float64       
 7   (artist, name)          106574 non-null  object        
 8   (set, split)            106574 non-null  object        
 9   (set, subset)           106574 non-null  category      
 10  (track, bit_rate)       106574 non-null  int64         
 11  (track, date_created)   106574 non-null  datetime64[ns]
 12  (track, duration)       106574

In [6]:
tracks.head()

album                          artist                \
         listens                 title favorites id   latitude   
track_id                                                         
2           6073  AWOL - A Way Of Life         9  1  40.058324   
3           6073  AWOL - A Way Of Life         9  1  40.058324   
5           6073  AWOL - A Way Of Life         9  1  40.058324   
10         47632     Constant Hitmaker        74  6        NaN   
20          2710                 Niris        10  4  51.895927   

                                                          set          \
                    location  longitude        name     split  subset   
track_id                                                                
2                 New Jersey -74.405661        AWOL  training   small   
3                 New Jersey -74.405661        AWOL  training  medium   
5                 New Jersey -74.405661        AWOL  training   small   
10                       NaN        NaN   Kurt Vile  training   small   
20        Colchester England   0.891874  Nicky Cook  training   large   

            track                                                              \
         bit_rate        date_created duration favorites genre_top     genres   
track_id                                                                        
2          256000 2008-11-26 01:48:12      168         2   Hip-Hop       [21]   
3          256000 2008-11-26 01:48:14      237         1   Hip-Hop       [21]   
5          256000 2008-11-26 01:48:20      206         6   Hip-Hop       [21]   
10         192000 2008-11-25 17:49:06      161       178       Pop       [10]   
20         256000 2008-11-26 01:48:56      311         0       NaN  [76, 103]   

                                                                             
                 genres_all interest language_code listens            title  
track_id                                                                     
2                      [21]     4656            en    1293             Food  
3                      [21]     1470            en     514     Electric Ave  
5                      [21]     1933            en    1151       This World  
10                     [10]    54881            en   50135          Freeway  
20        [17, 10, 76, 103]      978            en     361  Spiritual Level

In [7]:
genres

,#tracks,parent,title,top_level
genre_id,,,,
1,8693,38,Avant-Garde,38
2,5271,0,International,2
3,1752,0,Blues,3
4,4126,0,Jazz,4
5,4106,0,Classical,5
...,...,...,...,...
1032,60,102,Turkish,2
1060,30,46,Tango,2
1156,26,130,Fado,2


In [8]:
genres = genres.sort_values(by='#tracks', ascending=False)
genres

,#tracks,parent,title,top_level
genre_id,,,,
38,38154,0,Experimental,38
15,34413,0,Electronic,15
12,32923,0,Rock,12
1235,14938,0,Instrumental,1235
10,13845,0,Pop,10
...,...,...,...,...
493,4,651,Western Swing,9
173,4,86,N. Indian Traditional,2
377,1,19,Deep Funk,14


In [9]:
genresrock=genres.loc[genres['top_level']==12]
genresrock.sort_values(by='#tracks',ascending=False)

,#tracks,parent,title,top_level
genre_id,,,,
12,32923,0,Rock,12
25,9261,12,Punk,12
27,6041,12,Lo-Fi,12
66,5432,12,Indie-Rock,12
85,3548,12,Garage,12
58,2502,12,Psych-Rock,12
45,2469,12,Loud-Rock,12
70,2230,12,Industrial,12
53,2071,45,Noise-Rock,12


In [10]:
echonest.head()

echonest                                                    \
         audio_features                                                     
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476         0.010628  0.177647   
3              0.374408     0.528643  0.817461         0.001851  0.105880   
5              0.043567     0.745566  0.701470         0.000697  0.373143   
10             0.951670     0.658179  0.924525         0.965427  0.115474   
134            0.452217     0.513238  0.560410         0.019443  0.096567   

                                                                        \
                                           metadata                      
         speechiness    tempo   valence  album_date         album_name   
track_id                                                                 
2           0.159310  165.922  0.576661         NaN                NaN   
3           0.461818  126.957  0.269240         NaN                NaN   
5           0.124595  100.260  0.621661         NaN                NaN   
10          0.032985  111.562  0.963590  2008-03-11  Constant Hitmaker   
134         0.525519  114.290  0.894072         NaN                NaN   

                                                                             \
                                                                              
         artist_latitude       artist_location artist_longitude artist_name   
track_id                                                                      
2                32.6783           Georgia, US         -83.2230        AWOL   
3                32.6783           Georgia, US         -83.2230        AWOL   
5                32.6783           Georgia, US         -83.2230        AWOL   
10               39.9523  Philadelphia, PA, US         -75.1624   Kurt Vile   
134              32.6783           Georgia, US         -83.2230        AWOL   

                                                                              \
                                               ranks                           
                       release artist_discovery_rank artist_familiarity_rank   
track_id                                                                       
2         AWOL - A Way Of Life                   NaN                     NaN   
3         AWOL - A Way Of Life                   NaN                     NaN   
5         AWOL - A Way Of Life                   NaN                     NaN   
10           Constant Hitmaker                2635.0                  2544.0   
134       AWOL - A Way Of Life                   NaN                     NaN   

                                                                         \
                                                                          
         artist_hotttnesss_rank song_currency_rank song_hotttnesss_rank   
track_id                                                                  
2                           NaN                NaN                  NaN   
3                           NaN                NaN                  NaN   
5                           NaN                NaN                  NaN   
10                        397.0           115691.0              67609.0   
134                         NaN                NaN                  NaN   

                                                                              \
          social_features                                                      
         artist_discovery artist_familiarity artist_hotttnesss song_currency   
track_id                                                                       
2                0.388990           0.386740          0.406370      0.000000   
3                0.388990           0.386740          0.406370      0.000000   
5                0.388990           0.386740          0.406370   

In [11]:
echonest=echonest['echonest', 'audio_features']
echonest.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072


In [12]:
features.head()

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
20           -0.193837 -0.198527  0.201546  0.258556  0.775204  0.084794   

feature                                                                 \
statistics                                                               
number            07        08        09        10        11        12   
track_id                                                                 
2           1.481593  2.691455  0.866868  1.341231  1.347792  1.237658   
3           1.366848  1.054094  0.108103  0.619185  1.038253  1.292235   
5          -0.923192 -0.927232  0.666617  1.038546  0.268932  1.125141   
10          1.770694  1.604566  0.521217  1.982386  4.326824  1.300406   
20         -0.289294 -0.816410  0.043851 -0.804761 -0.990958 -0.430381   

feature                                                                 \
statistics       max                                                     
number            01        02        03        04        05        06   
track_id                                                                 
2           0.692500  0.569344  0.597041  0.625864  0.567330  0.443949   
3           0.677641  0.584248  0.581271  0.581182  0.454241  0.464841   
5           0.611014  0.651471  0.494528  0.448799  0.468624  0.454021   
10          0.461840  0.540411  0.446708  0.647553  0.591908  0.513306   
20          0.652864  0.676290  0.670288  0.598666  0.653607  0.697645   

feature                                                                 \
statistics                                                               
number            07        08        09        10        11        12   
track_id                                                                 
2           0.487976  0.497327  0.574435  0.579241  0.620102  0.586945   
3           0.542833  0.664720  0.511329  0.530998  0.603398  0.547428   
5           0.497172  0.559755  0.671287  0.610565  0.551663  0.603413   
10          0.651501  0.516887  0.511479  0.478263  0.638766  0.638495   
20          0.664929  0.686563  0.635117  0.667393  0.689589  0.683196   

feature                                                                 \
statistics      mean                                                     
number            01        02        03        04        05        06   
track_id                                                                 
2           0.474300  0.369816  0.236119  0.228068  0.222830  0.221415   
3           0.232784  0.229469  0.225674  0.216713  0.220512  0.242744   
5           0.258420  0.303385  0.250737  0.218562  0.245743  0.236018   
10          0.229882  0.286978  0.240096  0.226792  0.192443  0.288410   
20          0.202806  0.245125  0.262997  0.187961  0.182397  0.238173   

feature                                                                 \
statistics                                                               
number            07        08        09        10        11        12   
track_id                                                                 
2           0.229238  0.248795  0.196245  0.175809  0.200713  0.319972   
3           0.369235  0.420716  0.312129  0.242748  0.264292  0.225683   
5           0.275766  0.293982  0.346324  0.289821  0.246368  0.220939   
10          0.413348  0.349137  0.268424  0.243144  0.268941  0.236763   
20          0.278600  0.292905  0.247150  0.292501  0.3

In [13]:
indexlist=echonest.index.values.tolist()
indexlist

[2,
 3,
 5,
 10,
 134,
 139,
 140,
 141,
 142,
 144,
 145,
 146,
 147,
 153,
 154,
 155,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 226,
 228,
 236,
 237,
 238,
 247,
 249,
 252,
 255,
 256,
 257,
 258,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 359,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 384,
 386,
 387,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 410,
 411,
 412,
 414,
 416,
 461,
 462,
 463,
 464,
 465,
 520,
 521,
 522,
 523,
 524,
 525,
 526,
 527,
 528,
 529,
 530,
 531,
 532,
 533,

In [14]:
echonest.shape

(13129, 8)

In [15]:
tracks.tail()

album                                   artist                  \
         listens                          title favorites     id latitude   
track_id                                                                    
155316      1506  Live at Monty Hall, 2/17/2017         0  24357      NaN   
155317      1506  Live at Monty Hall, 2/17/2017         0  24357      NaN   
155318      1506  Live at Monty Hall, 2/17/2017         0  24357      NaN   
155319      1506  Live at Monty Hall, 2/17/2017         0  24357      NaN   
155320      7481      What I Tell Myself Vol. 2        12  21615      NaN   

                                                                    set  \
                       location longitude              name       split   
track_id                                                                  
155316               New Jersey       NaN           Spowder    training   
155317               New Jersey       NaN           Spowder    training   
155318               New Jersey       NaN           Spowder    training   
155319               New Jersey       NaN           Spowder    training   
155320    Jersey City, NJ 07302       NaN  Forget the Whale  validation   

                   track                                                   \
         subset bit_rate        date_created duration favorites genre_top   
track_id                                                                    
155316    large   320000 2017-03-30 15:23:34      162         1      Rock   
155317    large   320000 2017-03-30 15:23:36      217         1      Rock   
155318    large   320000 2017-03-30 15:23:37      404         2      Rock   
155319    large   320000 2017-03-30 15:23:39      146         0      Rock   
155320    large   320000 2017-03-30 09:15:36      198         1       NaN   

                                                                          \
                 genres        genres_all interest language_code listens   
track_id                                                                   
155316             [25]          [25, 12]      122           NaN     102   
155317             [25]          [25, 12]      194           NaN     165   
155318             [25]          [25, 12]      214           NaN     168   
155319             [25]          [25, 12]      336           NaN     294   
155320    [10, 12, 169]  [169, 10, 12, 9]      972           NaN     705   

                                                     
                                              title  
track_id                                             
155316                                    The Auger  
155317                              Let's Skin Ruby  
155318           My House Smells Like Kim Deal/Pulp  
155319                      The Man With Two Mouths  
155320    Another Trick Up My Sleeve (Instrumental)

In [16]:
tracks.shape

(106574, 21)

In [17]:
df=tracks
df.head()

album                          artist                \
         listens                 title favorites id   latitude   
track_id                                                         
2           6073  AWOL - A Way Of Life         9  1  40.058324   
3           6073  AWOL - A Way Of Life         9  1  40.058324   
5           6073  AWOL - A Way Of Life         9  1  40.058324   
10         47632     Constant Hitmaker        74  6        NaN   
20          2710                 Niris        10  4  51.895927   

                                                          set          \
                    location  longitude        name     split  subset   
track_id                                                                
2                 New Jersey -74.405661        AWOL  training   small   
3                 New Jersey -74.405661        AWOL  training  medium   
5                 New Jersey -74.405661        AWOL  training   small   
10                       NaN        NaN   Kurt Vile  training   small   
20        Colchester England   0.891874  Nicky Cook  training   large   

            track                                                              \
         bit_rate        date_created duration favorites genre_top     genres   
track_id                                                                        
2          256000 2008-11-26 01:48:12      168         2   Hip-Hop       [21]   
3          256000 2008-11-26 01:48:14      237         1   Hip-Hop       [21]   
5          256000 2008-11-26 01:48:20      206         6   Hip-Hop       [21]   
10         192000 2008-11-25 17:49:06      161       178       Pop       [10]   
20         256000 2008-11-26 01:48:56      311         0       NaN  [76, 103]   

                                                                             
                 genres_all interest language_code listens            title  
track_id                                                                     
2                      [21]     4656            en    1293             Food  
3                      [21]     1470            en     514     Electric Ave  
5                      [21]     1933            en    1151       This World  
10                     [10]    54881            en   50135          Freeway  
20        [17, 10, 76, 103]      978            en     361  Spiritual Level

In [18]:
df.columns

MultiIndex([( 'album',       'listens'),
            ( 'album',         'title'),
            ('artist',     'favorites'),
            ('artist',            'id'),
            ('artist',      'latitude'),
            ('artist',      'location'),
            ('artist',     'longitude'),
            ('artist',          'name'),
            (   'set',         'split'),
            (   'set',        'subset'),
            ( 'track',      'bit_rate'),
            ( 'track',  'date_created'),
            ( 'track',      'duration'),
            ( 'track',     'favorites'),
            ( 'track',     'genre_top'),
            ( 'track',        'genres'),
            ( 'track',    'genres_all'),
            ( 'track',      'interest'),
            ( 'track', 'language_code'),
            ( 'track',       'listens'),
            ( 'track',         'title')],
           )

In [19]:
tracks=df[df.index.isin(indexlist)]

In [20]:
tracks.shape

(13129, 21)

In [21]:
tracks.reset_index().rename({'index':'tracks_id'}, axis = 'columns')

track_id   album                          artist                    \
               listens                 title favorites     id   latitude   
0            2    6073  AWOL - A Way Of Life         9      1  40.058324   
1            3    6073  AWOL - A Way Of Life         9      1  40.058324   
2            5    6073  AWOL - A Way Of Life         9      1  40.058324   
3           10   47632     Constant Hitmaker        74      6        NaN   
4          134    6073  AWOL - A Way Of Life         9      1  40.058324   
...        ...     ...                   ...       ...    ...        ...   
13124   124857   55904  Do You Know The Word         8  20818        NaN   
13125   124862   55904  Do You Know The Word         8  20818        NaN   
13126   124863   55904  Do You Know The Word         8  20818        NaN   
13127   124864   55904  Do You Know The Word         8  20818        NaN   
13128   124911    6108   Suicide Beauty Girl        81  13530  35.670479   

                                                       set            track  \
           location   longitude             name     split  subset bit_rate   
0        New Jersey  -74.405661             AWOL  training   small   256000   
1        New Jersey  -74.405661             AWOL  training  medium   256000   
2        New Jersey  -74.405661             AWOL  training   small   256000   
3               NaN         NaN        Kurt Vile  training   small   192000   
4        New Jersey  -74.405661             AWOL  training  medium   256000   
...             ...         ...              ...       ...     ...      ...   
13124           NaN         NaN            BASIC  training   large   320000   
13125           NaN         NaN            BASIC  training   large   320000   
13126           NaN         NaN            BASIC  training   large   320000   
13127           NaN         NaN            BASIC  training   large   320000   
13128  Tokyo, Japan  139.740921  Yusuke Tsutsumi  training   large   320000   

                                                                    \
             date_created duration favorites genre_top      genres   
0     2008-11-26 01:48:12      168         2   Hip-Hop        [21]   
1     2008-11-26 01:48:14      237         1   Hip-Hop        [21]   
2     2008-11-26 01:48:20      206         6   Hip-Hop        [21]   
3     2008-11-25 17:49:06      161       178       Pop        [10]   
4     2008-11-26 01:43:19      207         3   Hip-Hop        [21]   
...                   ...      ...       ...       ...         ...   
13124 2015-09-09 14:40:09      268        11       NaN  [286, 542]   
13125 2015-09-09 14:56:06      244         3       NaN  [286, 542]   
13126 2015-09-09 14:56:06      382         2       NaN  [286, 542]   
13127 2015-09-09 14:56:08      290         5       NaN  [286, 542]   
13128 2015-09-11 12:10:09      346         9       NaN     [5, 15]   

                                                          \
               genres_all interest language_code listens   
0                    [21]     4656            en    1293   
1                    [21]     1470            en     514   
2                    [21]     1933            en    1151   
3                    [10]    54881            en   50135   
4                    [21]     1126            en     943   
...                   ...      ...           ...     ...   
13124  [542, 21, 286, 15]    20996           NaN   13365   
13125  [542, 21, 286, 15]     4501           NaN    3588   
13126  [542, 21, 286, 15]     3641           NaN    2956   
13127  [542, 21, 286, 15]     3960           NaN    3126   
13128             [5, 15]     7548           NaN    6108   

                                     
                              title  
0                              Food  
1                      Electric Ave  
2                        This World  
3                           Freeway  
4                      Street Music  
...                            

In [22]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13129 entries, 2 to 124911
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   (album, listens)        13129 non-null  int64         
 1   (album, title)          13129 non-null  object        
 2   (artist, favorites)     13129 non-null  int64         
 3   (artist, id)            13129 non-null  int64         
 4   (artist, latitude)      8052 non-null   float64       
 5   (artist, location)      9083 non-null   object        
 6   (artist, longitude)     8052 non-null   float64       
 7   (artist, name)          13129 non-null  object        
 8   (set, split)            13129 non-null  object        
 9   (set, subset)           13129 non-null  category      
 10  (track, bit_rate)       13129 non-null  int64         
 11  (track, date_created)   13129 non-null  datetime64[ns]
 12  (track, duration)       13129 non-null  int64

In [23]:
pd.options.mode.chained_assignment = None 
tracks[('album', 'title')]=tracks[('album', 'title')].astype(str)
tracks[('artist', 'location')]=tracks[('artist', 'location')].astype(str)
tracks[('artist', 'name')]=tracks[('artist', 'name')].astype(str)
tracks[('set', 'split')]=tracks[('set', 'split')].astype(str)
tracks[('set', 'subset')]=tracks[('set', 'subset')].astype(str)
tracks[('track', 'genre_top')]=tracks[('track', 'genre_top')].astype(str)
tracks[('track', 'genres')]=tracks[('track', 'genres')].astype(str)
tracks[('track', 'genres_all')]=tracks[('track', 'genres_all')].astype(str)
tracks[('track', 'language_code')]=tracks[('track', 'language_code')].astype(str)
tracks[('track', 'title')]=tracks[('track', 'title')].astype(str)

In [24]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13129 entries, 2 to 124911
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   (album, listens)        13129 non-null  int64         
 1   (album, title)          13129 non-null  object        
 2   (artist, favorites)     13129 non-null  int64         
 3   (artist, id)            13129 non-null  int64         
 4   (artist, latitude)      8052 non-null   float64       
 5   (artist, location)      13129 non-null  object        
 6   (artist, longitude)     8052 non-null   float64       
 7   (artist, name)          13129 non-null  object        
 8   (set, split)            13129 non-null  object        
 9   (set, subset)           13129 non-null  object        
 10  (track, bit_rate)       13129 non-null  int64         
 11  (track, date_created)   13129 non-null  datetime64[ns]
 12  (track, duration)       13129 non-null  int64

In [25]:
de=features
features=de[de.index.isin(indexlist)]

In [26]:
echonest.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072


In [27]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13129 entries, 2 to 124911
Columns: 518 entries, ('chroma_cens', 'kurtosis', '01') to ('zcr', 'std', '01')
dtypes: float64(518)
memory usage: 52.0 MB


In [28]:
result=pd.concat([tracks, echonest, features], axis=1)

In [29]:
result.head()

,"(album, listens)","(album, title)","(artist, favorites)","(artist, id)","(artist, latitude)","(artist, location)","(artist, longitude)","(artist, name)","(set, split)","(set, subset)","(track, bit_rate)","(track, date_created)","(track, duration)","(track, favorites)","(track, genre_top)","(track, genres)","(track, genres_all)","(track, interest)","(track, language_code)","(track, listens)","(track, title)",acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,"(chroma_cens, kurtosis, 01)","(chroma_cens, kurtosis, 02)","(chroma_cens, kurtosis, 03)","(chroma_cens, kurtosis, 04)","(chroma_cens, kurtosis, 05)","(chroma_cens, kurtosis, 06)","(chroma_cens, kurtosis, 07)","(chroma_cens, kurtosis, 08)","(chroma_cens, kurtosis, 09)","(chroma_cens, kurtosis, 10)","(chroma_cens, kurtosis, 11)","(chroma_cens, kurtosis, 12)","(chroma_cens, max, 01)","(chroma_cens, max, 02)","(chroma_cens, max, 03)","(chroma_cens, max, 04)","(chroma_cens, max, 05)","(chroma_cens, max, 06)","(chroma_cens, max, 07)","(chroma_cens, max, 08)","(chroma_cens, max, 09)","(chroma_cens, max, 10)","(chroma_cens, max, 11)","(chroma_cens, max, 12)","(chroma_cens, mean, 01)","(chroma_cens, mean, 02)","(chroma_cens, mean, 03)","(chroma_cens, mean, 04)","(chroma_cens, mean, 05)","(chroma_cens, mean, 06)","(chroma_cens, mean, 07)","(chroma_cens, mean, 08)","(chroma_cens, mean, 09)","(chroma_cens, mean, 10)","(chroma_cens, mean, 11)","(chroma_cens, mean, 12)","(chroma_cens, median, 01)","(chroma_cens, median, 02)","(chroma_cens, median, 03)","(chroma_cens, median, 04)","(chroma_cens, median, 05)","(chroma_cens, median, 06)","(chroma_cens, median, 07)","(chroma_cens, median, 08)","(chroma_cens, median, 09)","(chroma_cens, median, 10)","(chroma_cens, median, 11)","(chroma_cens, median, 12)","(chroma_cens, min, 01)","(chroma_cens, min, 02)","(chroma_cens, min, 03)","(chroma_cens, min, 04)","(chroma_cens, min, 05)","(chroma_cens, min, 06)","(chroma_cens, min, 07)","(chroma_cens, min, 08)","(chroma_cens, min, 09)","(chroma_cens, min, 10)","(chroma_cens, min, 11)","(chroma_cens, min, 12)","(chroma_cens, skew, 01)","(chroma_cens, skew, 02)","(chroma_cens, skew, 03)","(chroma_cens, skew, 04)","(chroma_cens, skew, 05)","(chroma_cens, skew, 06)","(chroma_cens, skew, 07)","(chroma_cens, skew, 08)","(chroma_cens, skew, 09)","(chroma_cens, skew, 10)","(chroma_cens, skew, 11)","(chroma_cens, skew, 12)","(chroma_cens, std, 01)","(chroma_cens, std, 02)","(chroma_cens, std, 03)","(chroma_cens, std, 04)","(chroma_cens, std, 05)","(chroma_cens, std, 06)","(chroma_cens, std, 07)","(chroma_cens, std, 08)","(chroma_cens, std, 09)","(chroma_cens, std, 10)","(chroma_cens, std, 11)","(chroma_cens, std, 12)","(chroma_cqt, kurtosis, 01)","(chroma_cqt, kurtosis, 02)","(chroma_cqt, kurtosis, 03)","(chroma_cqt, kurtosis, 04)","(chroma_cqt, kurtosis, 05)","(chroma_cqt, kurtosis, 06)","(chroma_cqt, kurtosis, 07)","(chroma_cqt, kurtosis, 08)","(chroma_cqt, kurtosis, 09)","(chroma_cqt, kurtosis, 10)","(chroma_cqt, kurtosis, 11)","(chroma_cqt, kurtosis, 12)","(chroma_cqt, max, 01)","(chroma_cqt, max, 02)","(chroma_cqt, max, 03)","(chroma_cqt, max, 04)","(chroma_cqt, max, 05)","(chroma_cqt, max, 06)","(chroma_cqt, max, 07)","(chroma_cqt, max, 08)","(chroma_cqt, max, 09)","(chroma_cqt, max, 10)","(chroma_cqt, max, 11)","(chroma_cqt, max, 12)","(chroma_cqt, mean, 01)","(chroma_cqt, mean, 02)","(chroma_cqt, mean, 03)","(chroma_cqt, mean, 04)","(chroma_cqt, mean, 05)","(chroma_cqt, mean, 06)","(chroma_cqt, mean, 07)","(chroma_cqt, mean, 08)","(chroma_cqt, mean, 09)","(chroma_cqt, mean, 10)","(chroma_cqt, mean, 11)","(chroma_cqt, mean, 12)","(chroma_cqt, median, 01)","(chroma_cqt, median, 02)","(chroma_cqt, median, 03)","(chroma_cqt, median, 04)","(chroma_cqt, median, 05)","(chroma_cqt, median, 06)","(chroma_cqt, median, 07)","(chroma_cqt, median, 08)","(chroma_cqt, median, 09)","(chroma_cqt, median, 10)","(chroma_cqt, median, 11)","(chroma_cqt, median, 12)","(chroma_cqt, min, 01)

In [30]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13129 entries, 2 to 124911
Columns: 547 entries, ('album', 'listens') to ('zcr', 'std', '01')
dtypes: datetime64[ns](1), float64(528), int64(8), object(10)
memory usage: 54.9+ MB


In [31]:
result[('set', 'split')].value_counts()

training      10728
test           1293
validation     1108
Name: (set, split), dtype: int64

In [32]:
result.to_csv('result.csv')

In [33]:
result = pd.read_csv('result.csv', index_col='track_id')  
result.head()

,"('album', 'listens')","('album', 'title')","('artist', 'favorites')","('artist', 'id')","('artist', 'latitude')","('artist', 'location')","('artist', 'longitude')","('artist', 'name')","('set', 'split')","('set', 'subset')","('track', 'bit_rate')","('track', 'date_created')","('track', 'duration')","('track', 'favorites')","('track', 'genre_top')","('track', 'genres')","('track', 'genres_all')","('track', 'interest')","('track', 'language_code')","('track', 'listens')","('track', 'title')",acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,"('chroma_cens', 'kurtosis', '01')","('chroma_cens', 'kurtosis', '02')","('chroma_cens', 'kurtosis', '03')","('chroma_cens', 'kurtosis', '04')","('chroma_cens', 'kurtosis', '05')","('chroma_cens', 'kurtosis', '06')","('chroma_cens', 'kurtosis', '07')","('chroma_cens', 'kurtosis', '08')","('chroma_cens', 'kurtosis', '09')","('chroma_cens', 'kurtosis', '10')","('chroma_cens', 'kurtosis', '11')","('chroma_cens', 'kurtosis', '12')","('chroma_cens', 'max', '01')","('chroma_cens', 'max', '02')","('chroma_cens', 'max', '03')","('chroma_cens', 'max', '04')","('chroma_cens', 'max', '05')","('chroma_cens', 'max', '06')","('chroma_cens', 'max', '07')","('chroma_cens', 'max', '08')","('chroma_cens', 'max', '09')","('chroma_cens', 'max', '10')","('chroma_cens', 'max', '11')","('chroma_cens', 'max', '12')","('chroma_cens', 'mean', '01')","('chroma_cens', 'mean', '02')","('chroma_cens', 'mean', '03')","('chroma_cens', 'mean', '04')","('chroma_cens', 'mean', '05')","('chroma_cens', 'mean', '06')","('chroma_cens', 'mean', '07')","('chroma_cens', 'mean', '08')","('chroma_cens', 'mean', '09')","('chroma_cens', 'mean', '10')","('chroma_cens', 'mean', '11')","('chroma_cens', 'mean', '12')","('chroma_cens', 'median', '01')","('chroma_cens', 'median', '02')","('chroma_cens', 'median', '03')","('chroma_cens', 'median', '04')","('chroma_cens', 'median', '05')","('chroma_cens', 'median', '06')","('chroma_cens', 'median', '07')","('chroma_cens', 'median', '08')","('chroma_cens', 'median', '09')","('chroma_cens', 'median', '10')","('chroma_cens', 'median', '11')","('chroma_cens', 'median', '12')","('chroma_cens', 'min', '01')","('chroma_cens', 'min', '02')","('chroma_cens', 'min', '03')","('chroma_cens', 'min', '04')","('chroma_cens', 'min', '05')","('chroma_cens', 'min', '06')","('chroma_cens', 'min', '07')","('chroma_cens', 'min', '08')","('chroma_cens', 'min', '09')","('chroma_cens', 'min', '10')","('chroma_cens', 'min', '11')","('chroma_cens', 'min', '12')","('chroma_cens', 'skew', '01')","('chroma_cens', 'skew', '02')","('chroma_cens', 'skew', '03')","('chroma_cens', 'skew', '04')","('chroma_cens', 'skew', '05')","('chroma_cens', 'skew', '06')","('chroma_cens', 'skew', '07')","('chroma_cens', 'skew', '08')","('chroma_cens', 'skew', '09')","('chroma_cens', 'skew', '10')","('chroma_cens', 'skew', '11')","('chroma_cens', 'skew', '12')","('chroma_cens', 'std', '01')","('chroma_cens', 'std', '02')","('chroma_cens', 'std', '03')","('chroma_cens', 'std', '04')","('chroma_cens', 'std', '05')","('chroma_cens', 'std', '06')","('chroma_cens', 'std', '07')","('chroma_cens', 'std', '08')","('chroma_cens', 'std', '09')","('chroma_cens', 'std', '10')","('chroma_cens', 'std', '11')","('chroma_cens', 'std', '12')","('chroma_cqt', 'kurtosis', '01')","('chroma_cqt', 'kurtosis', '02')","('chroma_cqt', 'kurtosis', '03')","('chroma_cqt', 'kurtosis', '04')","('chroma_cqt', 'kurtosis', '05')","('chroma_cqt', 'kurtosis', '06')","('chroma_cqt', 'kurtosis', '07')","('chroma_cqt', 'kurtosis', '08')","('chroma_cqt', 'kurtosis', '09')","('chroma_cqt', 'kurtosis', '10')","('chroma_cqt', 'kurtosis', '11')","('chroma_cqt', 'kurtosis', '12')","('chroma_cqt', 'max', '01')","('chroma_cqt', 'max', '02')","('chroma_cqt', 'max', '03')","('chroma_cqt', 'max', '04')","('chroma_cqt', 'max', '05')","('chroma_cqt', 'max', '06')","('chroma_cqt', 'max', '07')","('chroma_cqt', 'max', '08')","('chroma_cqt', 'max', '09')","('c

In [34]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13129 entries, 2 to 124911
Columns: 547 entries, ('album', 'listens') to ('zcr', 'std', '01')
dtypes: float64(528), int64(8), object(11)
memory usage: 54.9+ MB


In [35]:
result=result.drop(columns=["('album', 'title')"])
result=result.drop(columns=["('artist', 'id')"])
result=result.drop(columns=["('artist', 'name')"])
result=result.drop(columns=["('artist', 'location')"])
result=result.drop(columns=["('set', 'subset')"])
result=result.drop(columns=["('track', 'language_code')"])
result=result.drop(columns=["('track', 'title')"])
result=result.drop(columns=["('track', 'date_created')"])
result=result.drop(columns=["('artist', 'latitude')"])
result=result.drop(columns=["('artist', 'longitude')"])

In [36]:
result.head()

,"('album', 'listens')","('artist', 'favorites')","('set', 'split')","('track', 'bit_rate')","('track', 'duration')","('track', 'favorites')","('track', 'genre_top')","('track', 'genres')","('track', 'genres_all')","('track', 'interest')","('track', 'listens')",acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,"('chroma_cens', 'kurtosis', '01')","('chroma_cens', 'kurtosis', '02')","('chroma_cens', 'kurtosis', '03')","('chroma_cens', 'kurtosis', '04')","('chroma_cens', 'kurtosis', '05')","('chroma_cens', 'kurtosis', '06')","('chroma_cens', 'kurtosis', '07')","('chroma_cens', 'kurtosis', '08')","('chroma_cens', 'kurtosis', '09')","('chroma_cens', 'kurtosis', '10')","('chroma_cens', 'kurtosis', '11')","('chroma_cens', 'kurtosis', '12')","('chroma_cens', 'max', '01')","('chroma_cens', 'max', '02')","('chroma_cens', 'max', '03')","('chroma_cens', 'max', '04')","('chroma_cens', 'max', '05')","('chroma_cens', 'max', '06')","('chroma_cens', 'max', '07')","('chroma_cens', 'max', '08')","('chroma_cens', 'max', '09')","('chroma_cens', 'max', '10')","('chroma_cens', 'max', '11')","('chroma_cens', 'max', '12')","('chroma_cens', 'mean', '01')","('chroma_cens', 'mean', '02')","('chroma_cens', 'mean', '03')","('chroma_cens', 'mean', '04')","('chroma_cens', 'mean', '05')","('chroma_cens', 'mean', '06')","('chroma_cens', 'mean', '07')","('chroma_cens', 'mean', '08')","('chroma_cens', 'mean', '09')","('chroma_cens', 'mean', '10')","('chroma_cens', 'mean', '11')","('chroma_cens', 'mean', '12')","('chroma_cens', 'median', '01')","('chroma_cens', 'median', '02')","('chroma_cens', 'median', '03')","('chroma_cens', 'median', '04')","('chroma_cens', 'median', '05')","('chroma_cens', 'median', '06')","('chroma_cens', 'median', '07')","('chroma_cens', 'median', '08')","('chroma_cens', 'median', '09')","('chroma_cens', 'median', '10')","('chroma_cens', 'median', '11')","('chroma_cens', 'median', '12')","('chroma_cens', 'min', '01')","('chroma_cens', 'min', '02')","('chroma_cens', 'min', '03')","('chroma_cens', 'min', '04')","('chroma_cens', 'min', '05')","('chroma_cens', 'min', '06')","('chroma_cens', 'min', '07')","('chroma_cens', 'min', '08')","('chroma_cens', 'min', '09')","('chroma_cens', 'min', '10')","('chroma_cens', 'min', '11')","('chroma_cens', 'min', '12')","('chroma_cens', 'skew', '01')","('chroma_cens', 'skew', '02')","('chroma_cens', 'skew', '03')","('chroma_cens', 'skew', '04')","('chroma_cens', 'skew', '05')","('chroma_cens', 'skew', '06')","('chroma_cens', 'skew', '07')","('chroma_cens', 'skew', '08')","('chroma_cens', 'skew', '09')","('chroma_cens', 'skew', '10')","('chroma_cens', 'skew', '11')","('chroma_cens', 'skew', '12')","('chroma_cens', 'std', '01')","('chroma_cens', 'std', '02')","('chroma_cens', 'std', '03')","('chroma_cens', 'std', '04')","('chroma_cens', 'std', '05')","('chroma_cens', 'std', '06')","('chroma_cens', 'std', '07')","('chroma_cens', 'std', '08')","('chroma_cens', 'std', '09')","('chroma_cens', 'std', '10')","('chroma_cens', 'std', '11')","('chroma_cens', 'std', '12')","('chroma_cqt', 'kurtosis', '01')","('chroma_cqt', 'kurtosis', '02')","('chroma_cqt', 'kurtosis', '03')","('chroma_cqt', 'kurtosis', '04')","('chroma_cqt', 'kurtosis', '05')","('chroma_cqt', 'kurtosis', '06')","('chroma_cqt', 'kurtosis', '07')","('chroma_cqt', 'kurtosis', '08')","('chroma_cqt', 'kurtosis', '09')","('chroma_cqt', 'kurtosis', '10')","('chroma_cqt', 'kurtosis', '11')","('chroma_cqt', 'kurtosis', '12')","('chroma_cqt', 'max', '01')","('chroma_cqt', 'max', '02')","('chroma_cqt', 'max', '03')","('chroma_cqt', 'max', '04')","('chroma_cqt', 'max', '05')","('chroma_cqt', 'max', '06')","('chroma_cqt', 'max', '07')","('chroma_cqt', 'max', '08')","('chroma_cqt', 'max', '09')","('chroma_cqt', 'max', '10')","('chroma_cqt', 'max', '11')","('chroma_cqt', 'max', '12')","('chroma_cqt', 'mean', '01')","('chroma_cqt', 'mean', '02')","('chroma_cqt', 'mean', '03')","('chroma_cqt', 'mean', '04')","('chroma_cqt', 'mean', '

In [37]:
list(result.columns)

["('album', 'listens')",
 "('artist', 'favorites')",
 "('set', 'split')",
 "('track', 'bit_rate')",
 "('track', 'duration')",
 "('track', 'favorites')",
 "('track', 'genre_top')",
 "('track', 'genres')",
 "('track', 'genres_all')",
 "('track', 'interest')",
 "('track', 'listens')",
 'acousticness',
 'danceability',
 'energy',
 'instrumentalness',
 'liveness',
 'speechiness',
 'tempo',
 'valence',
 "('chroma_cens', 'kurtosis', '01')",
 "('chroma_cens', 'kurtosis', '02')",
 "('chroma_cens', 'kurtosis', '03')",
 "('chroma_cens', 'kurtosis', '04')",
 "('chroma_cens', 'kurtosis', '05')",
 "('chroma_cens', 'kurtosis', '06')",
 "('chroma_cens', 'kurtosis', '07')",
 "('chroma_cens', 'kurtosis', '08')",
 "('chroma_cens', 'kurtosis', '09')",
 "('chroma_cens', 'kurtosis', '10')",
 "('chroma_cens', 'kurtosis', '11')",
 "('chroma_cens', 'kurtosis', '12')",
 "('chroma_cens', 'max', '01')",
 "('chroma_cens', 'max', '02')",
 "('chroma_cens', 'max', '03')",
 "('chroma_cens', 'max', '04')",
 "('chroma_c

In [38]:
result.rename({"('track', 'genre_top')": 'genre'}, axis=1, inplace=True)
result.rename({"('track', 'genres_all')": 'all'}, axis=1, inplace=True)

In [39]:
list(result.columns)


["('album', 'listens')",
 "('artist', 'favorites')",
 "('set', 'split')",
 "('track', 'bit_rate')",
 "('track', 'duration')",
 "('track', 'favorites')",
 'genre',
 "('track', 'genres')",
 'all',
 "('track', 'interest')",
 "('track', 'listens')",
 'acousticness',
 'danceability',
 'energy',
 'instrumentalness',
 'liveness',
 'speechiness',
 'tempo',
 'valence',
 "('chroma_cens', 'kurtosis', '01')",
 "('chroma_cens', 'kurtosis', '02')",
 "('chroma_cens', 'kurtosis', '03')",
 "('chroma_cens', 'kurtosis', '04')",
 "('chroma_cens', 'kurtosis', '05')",
 "('chroma_cens', 'kurtosis', '06')",
 "('chroma_cens', 'kurtosis', '07')",
 "('chroma_cens', 'kurtosis', '08')",
 "('chroma_cens', 'kurtosis', '09')",
 "('chroma_cens', 'kurtosis', '10')",
 "('chroma_cens', 'kurtosis', '11')",
 "('chroma_cens', 'kurtosis', '12')",
 "('chroma_cens', 'max', '01')",
 "('chroma_cens', 'max', '02')",
 "('chroma_cens', 'max', '03')",
 "('chroma_cens', 'max', '04')",
 "('chroma_cens', 'max', '05')",
 "('chroma_cens'

In [40]:
result.genre.value_counts()


Rock                   3892
Electronic             2170
Hip-Hop                 910
Folk                    874
Old-Time / Historic     357
Pop                     346
Classical               265
Jazz                    241
International           133
Instrumental             84
Blues                    66
Experimental             17
Name: genre, dtype: int64

In [41]:
result.loc[result['all'] == "[2, 92, 15]", 'genre'] = "International"
result.loc[result['all'] == "[3, 4, 37]", 'genre'] = "Blues"
result.loc[result['all'] == "[8, 2]", 'genre'] = "International"
result.loc[result['all'] == "[25, 10, 12]", 'genre'] = "Pop"
result.loc[result['all'] == "[5, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[17, 58, 12, 33]", 'genre'] = "Folk"
result.loc[result['all'] == "[2, 4, 118]", 'genre'] = "International"
result.loc[result['all'] == "[66, 12, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[2, 171, 14, 15, 19]", 'genre'] = "International"
result.loc[result['all'] == "[2, 15]", 'genre'] = "International"
result.loc[result['all'] == "[10, 12, 66]", 'genre'] = "Pop"
result.loc[result['all'] == "[17, 10, 12]", 'genre'] = "Pop"
result.loc[result['all'] == "[12, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[74, 3, 4]", 'genre'] = "Blues"
result.loc[result['all'] == "[17, 12, 25]", 'genre'] = "Folk"
result.loc[result['all'] == "[17, 12]", 'genre'] = "Folk"
result.loc[result['all'] == "[17, 10]", 'genre'] = "Pop"
result.loc[result['all'] == "[65, 12, 20, 166]", 'genre'] = "Rock"
result.loc[result['all'] == "[137, 12, 9]", 'genre'] = "Rock"
result.loc[result['all'] == "[10, 12]", 'genre'] = "Pop"
result.loc[result['all'] == "[89, 12, 25, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[8, 441, 5]", 'genre'] = "Classical"
result.loc[result['all'] == "[71, 362, 10, 12, 25]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 172, 58, 12]", 'genre'] = "International"
result.loc[result['all'] == "[2, 58, 172, 12]", 'genre'] = "International"
result.loc[result['all'] == "[33, 2, 102, 12, 17, 58]", 'genre'] = "Folk"
result.loc[result['all'] == "[58, 2, 12, 92]", 'genre'] = "International"
result.loc[result['all'] == "[58, 2, 12, 102]", 'genre'] = "International"
result.loc[result['all'] == "[17, 2]", 'genre'] = "Folk"
result.loc[result['all'] == "[70, 71, 12, 15, 25]", 'genre'] = "Electronic"
result.loc[result['all'] == "[17, 10, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 130, 4, 74, 118]", 'genre'] = "International"
result.loc[result['all'] == "[17, 66, 12]", 'genre'] = "Folk"
result.loc[result['all'] == "[10, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[17, 10, 12, 66]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 58, 12, 79]", 'genre'] = "International"
result.loc[result['all'] == "[58, 2, 12, 79]", 'genre'] = "International"
result.loc[result['all'] == "[2, 58, 12, 130]", 'genre'] = "International"
result.loc[result['all'] == "[2, 169, 9, 12, 58]", 'genre'] = "International"
result.loc[result['all'] == "[2, 58, 12, 102]", 'genre'] = "International"
result.loc[result['all'] == "[33, 2, 12, 17, 58]", 'genre'] = "Folk"
result.loc[result['all'] == "[2, 58, 12]", 'genre'] = "International"
result.loc[result['all'] == "[232, 17, 2]", 'genre'] = "Folk"
result.loc[result['all'] == "[2, 4, 38, 74, 250]", 'genre'] = "Experimental"
result.loc[result['all'] == "[21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[17, 137, 9]", 'genre'] = "Folk"
result.loc[result['all'] == "[6, 38, 8, 16, 763]", 'genre'] = "Experimental"
result.loc[result['all'] == "[2, 79, 15]", 'genre'] = "International"
result.loc[result['all'] == "[17, 9, 63]", 'genre'] = "Folk"
result.loc[result['all'] == "[33, 58, 12, 17]", 'genre'] = "Folk"
result.loc[result['all'] == "[33, 26, 12, 17, 113, 58]", 'genre'] = "Folk"
result.loc[result['all'] == "[64, 25, 12, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[17, 2, 171]", 'genre'] = "Folk"
result.loc[result['all'] == "[66, 296, 12, 15, 89, 25]", 'genre'] = "Electronic"
result.loc[result['all'] == "[2, 19, 21, 14]", 'genre'] = "International"
result.loc[result['all'] == "[26, 58, 38, 12, 113, 250]", 'genre'] = "Experimental"
result.loc[result['all'] == "[250, 4, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[58, 250, 12, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[296, 10, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[13, 15, 18, 1235, 311]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[4, 15]", 'genre'] = "Jazz"
result.loc[result['all'] == "[10, 19, 21, 14]", 'genre'] = "Pop"
result.loc[result['all'] == "[12, 14]", 'genre'] = "Rock"
result.loc[result['all'] == "[8, 17]", 'genre'] = "Old-Time / Historic"
result.loc[result['all'] == "[17, 3]", 'genre'] = "Blues"
result.loc[result['all'] == "[6, 38, 16, 17, 763]", 'genre'] = "Experimental"
result.loc[result['all'] == "[297, 58, 12, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[2, 3, 79, 15]", 'genre'] = "Blues"
result.loc[result['all'] == "[4, 12]", 'genre'] = "Jazz"
result.loc[result['all'] == "[41, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[10, 26, 12]", 'genre'] = "Pop"
result.loc[result['all'] == "[41, 286, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[4, 21]", 'genre'] = "Jazz"
result.loc[result['all'] == "[2, 179, 4, 118]", 'genre'] = "International"
result.loc[result['all'] == "[31, 4, 12, 15]", 'genre'] = "Jazz"
result.loc[result['all'] == "[17, 12, 33]", 'genre'] = "Folk"
result.loc[result['all'] == "[12, 70, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[10, 13]", 'genre'] = "Pop"
result.loc[result['all'] == "[89, 10, 12, 25]", 'genre'] = "Pop"
result.loc[result['all'] == "[17, 4]", 'genre'] = "Jazz"
result.loc[result['all'] == "[66, 4, 36, 12, 88, 26]", 'genre'] = "Jazz"
result.loc[result['all'] == "[64, 12, 14, 15, 19, 25]", 'genre'] = "Electronic"
result.loc[result['all'] == "[250, 12, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[10, 2, 130]", 'genre'] = "International"
result.loc[result['all'] == "[9, 10, 137, 4]", 'genre'] = "Jazz"
result.loc[result['all'] == "[47, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[33, 12, 17, 359]", 'genre'] = "Folk"
result.loc[result['all'] == "[10, 11, 14, 362]", 'genre'] = "Pop"
result.loc[result['all'] == "[38, 33, 17, 47]", 'genre'] = "Experimental"
result.loc[result['all'] == "[17, 38, 41]", 'genre'] = "Experimental"
result.loc[result['all'] == "[8, 3]", 'genre'] = "Blues"
result.loc[result['all'] == "[2, 130, 4, 37, 10]", 'genre'] = "International"
result.loc[result['all'] == "[247, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[184, 41, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[100, 21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[100, 4, 21, 15]", 'genre'] = "Jazz"
result.loc[result['all'] == "[2, 171, 12, 15, 17]", 'genre'] = "Folk"
result.loc[result['all'] == "[13, 15, 400, 401, 181, 182, 311]", 'genre'] = "Electronic"
result.loc[result['all'] == "[2, 10, 362]", 'genre'] = "International"
result.loc[result['all'] == "[4, 5, 38, 74, 250, 187]", 'genre'] = "Experimental"
result.loc[result['all'] == "[240, 297, 10, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 79, 46, 15]", 'genre'] = "International"
result.loc[result['all'] == "[10, 36, 12]", 'genre'] = "Pop"
result.loc[result['all'] == "[89, 25, 3, 12]", 'genre'] = "Blues"
result.loc[result['all'] == "[3, 4]", 'genre'] = "Blues"
result.loc[result['all'] == "[2, 171, 4]", 'genre'] = "International"
result.loc[result['all'] == "[58, 10, 12, 362]", 'genre'] = "Pop"
result.loc[result['all'] == "[362, 10, 12, 15, 89, 25]", 'genre'] = "Pop"
result.loc[result['all'] == "[8, 4, 37]", 'genre'] = "Jazz"
result.loc[result['all'] == "[2, 12, 13, 311, 88]", 'genre'] = "International"
result.loc[result['all'] == "[10, 362, 3]", 'genre'] = "Blues"
result.loc[result['all'] == "[10, 3, 12, 66]", 'genre'] = "Blues"
result.loc[result['all'] == "[2, 12, 98, 31]", 'genre'] = "International"
result.loc[result['all'] == "[4, 5, 12, 440, 441, 31]", 'genre'] = "Jazz"
result.loc[result['all'] == "[17, 15]", 'genre'] = "Folk"
result.loc[result['all'] == "[4, 13, 14, 311]", 'genre'] = "Jazz"
result.loc[result['all'] == "[66, 19, 12, 14]", 'genre'] = "Rock"
result.loc[result['all'] == "[2, 4, 9, 171, 17, 63]", 'genre'] = "Folk"
result.loc[result['all'] == "[456, 33, 17, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[2, 468, 79, 15]", 'genre'] = "International"
result.loc[result['all'] == "[2, 38, 102, 15, 186]", 'genre'] = "Experimental"
result.loc[result['all'] == "[297, 12, 15, 31]", 'genre'] = "Electronic"
result.loc[result['all'] == "[468, 21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[240, 297, 12, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[2, 10, 15]", 'genre'] = "International"
result.loc[result['all'] == "[2, 3, 92]", 'genre'] = "Blues"
result.loc[result['all'] == "[38, 6, 360, 362, 10, 15, 1235]", 'genre'] = "Experimental"
result.loc[result['all'] == "[2, 468, 15]", 'genre'] = "International"
result.loc[result['all'] == "[17, 2, 79, 15]", 'genre'] = "Folk"
result.loc[result['all'] == "[811, 21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[17, 2, 118]", 'genre'] = "Folk"
result.loc[result['all'] == "[17, 66, 12, 33]", 'genre'] = "Folk"
result.loc[result['all'] == "[21, 182, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[17, 2, 102]", 'genre'] = "Folk"
result.loc[result['all'] == "[2, 21, 102, 15]", 'genre'] = "International"
result.loc[result['all'] == "[2, 21, 15]", 'genre'] = "International"
result.loc[result['all'] == "[2, 21, 46, 502]", 'genre'] = "International"
result.loc[result['all'] == "[9, 3]", 'genre'] = "Blues"
result.loc[result['all'] == "[3, 12]", 'genre'] = "Blues"
result.loc[result['all'] == "[6, 38, 12, 16, 58, 763]", 'genre'] = "Experimental"
result.loc[result['all'] == "[10, 12, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 46, 14, 17, 19]", 'genre'] = "Folk"
result.loc[result['all'] == "[17, 10, 103]", 'genre'] = "Pop"
result.loc[result['all'] == "[18, 1235, 15]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[19, 14, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[2, 77, 21]", 'genre'] = "International"
result.loc[result['all'] == "[187, 130, 2, 5]", 'genre'] = "Classical"
result.loc[result['all'] == "[130, 2, 5, 443, 187]", 'genre'] = "International"
result.loc[result['all'] == "[465, 2, 20, 130]", 'genre'] = "International"
result.loc[result['all'] == "[2, 5, 322, 130]", 'genre'] = "International"
result.loc[result['all'] == "[2, 187, 5, 130]", 'genre'] = "International"
result.loc[result['all'] == "[187, 18, 1235, 5]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[2, 12, 46]", 'genre'] = "International"
result.loc[result['all'] == "[2, 171, 12]", 'genre'] = "International"
result.loc[result['all'] == "[38, 2, 6]", 'genre'] = "Experimental"
result.loc[result['all'] == "[404, 2, 12, 85]", 'genre'] = "International"
result.loc[result['all'] == "[2, 12, 77]", 'genre'] = "International"
result.loc[result['all'] == "[2, 38, 6, 12, 404, 85]", 'genre'] = "Experimental"
result.loc[result['all'] == "[2, 12]", 'genre'] = "International"
result.loc[result['all'] == "[2, 77, 15, 401, 181]", 'genre'] = "International"
result.loc[result['all'] == "[2, 15, 21, 117, 539]", 'genre'] = "International"
result.loc[result['all'] == "[26, 12, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[64, 2, 297, 12, 77, 15, 25]", 'genre'] = "International"
result.loc[result['all'] == "[64, 2, 297, 12, 77, 15, 240, 25]", 'genre'] = "International"
result.loc[result['all'] == "[66, 10, 12, 362]", 'genre'] = "Pop"
result.loc[result['all'] == "[70, 38, 297, 12, 47, 15, 88, 89, 25]", 'genre'] = "Experimental"
result.loc[result['all'] == "[66, 10, 12, 111, 25]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 13, 46, 15, 311]", 'genre'] = "International"
result.loc[result['all'] == "[2, 46, 15]", 'genre'] = "International"
result.loc[result['all'] == "[41, 468, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[97, 4, 15]", 'genre'] = "Jazz"
result.loc[result['all'] == "[81, 2, 92, 15]", 'genre'] = "International"
result.loc[result['all'] == "[2, 102, 15]", 'genre'] = "International"
result.loc[result['all'] == "[2, 4, 46]", 'genre'] = "International"
result.loc[result['all'] == "[456, 5, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[33, 66, 12, 17]", 'genre'] = "Folk"
result.loc[result['all'] == "[17, 130, 2]", 'genre'] = "Folk"
result.loc[result['all'] == "[66, 12, 15, 113, 26]", 'genre'] = "Electronic"
result.loc[result['all'] == "[322, 5, 38, 456, 17]", 'genre'] = "Experimental"
result.loc[result['all'] == "[97, 4, 38, 41, 74, 15, 250]", 'genre'] = "Experimental"
result.loc[result['all'] == "[38, 6, 360, 15, 400, 182]", 'genre'] = "Experimental"
result.loc[result['all'] == "[2, 297, 79, 15, 468]", 'genre'] = "International"
result.loc[result['all'] == "[2, 286, 79, 15]", 'genre'] = "International"
result.loc[result['all'] == "[2, 100, 79, 21, 602]", 'genre'] = "International"
result.loc[result['all'] == "[2, 100, 79, 17, 21]", 'genre'] = "Folk"
result.loc[result['all'] == "[362, 10, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[66, 2, 12, 92]", 'genre'] = "International"
result.loc[result['all'] == "[10, 12, 15, 88, 314]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 66, 58, 12]", 'genre'] = "International"
result.loc[result['all'] == "[17, 10, 2, 117]", 'genre'] = "Pop"
result.loc[result['all'] == "[66, 362, 10, 12, 13, 311]", 'genre'] = "Pop"
result.loc[result['all'] == "[362, 10, 12]", 'genre'] = "Pop"
result.loc[result['all'] == "[13, 311, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[2, 130, 619, 12, 15]", 'genre'] = "International"
result.loc[result['all'] == "[15, 21, 182, 181]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[693, 21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[17, 4, 15]", 'genre'] = "Jazz"
result.loc[result['all'] == "[4, 13, 15, 311, 286]", 'genre'] = "Jazz"
result.loc[result['all'] == "[137, 10, 12, 9]", 'genre'] = "Pop"
result.loc[result['all'] == "[17, 26, 12, 15]", 'genre'] = "Folk"
result.loc[result['all'] == "[5, 38, 456, 15, 187]", 'genre'] = "Experimental"
result.loc[result['all'] == "[2, 172, 17, 18, 1235]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[296, 2, 92, 15]", 'genre'] = "International"
result.loc[result['all'] == "[2, 361, 12, 109, 89, 25]", 'genre'] = "International"
result.loc[result['all'] == "[66, 297, 12, 15, 240]", 'genre'] = "Electronic"
result.loc[result['all'] == "[322, 4, 5, 137, 9, 17]", 'genre'] = "Jazz"
result.loc[result['all'] == "[6, 38, 8, 137, 360, 9]", 'genre'] = "Experimental"
result.loc[result['all'] == "[10, 12, 66, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[296, 468, 21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[17, 2, 10, 171]", 'genre'] = "Folk"
result.loc[result['all'] == "[18, 1235, 12, 15]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[66, 4, 12]", 'genre'] = "Jazz"
result.loc[result['all'] == "[2, 21, 79]", 'genre'] = "International"
result.loc[result['all'] == "[2, 171, 21]", 'genre'] = "International"
result.loc[result['all'] == "[2, 296, 171, 15, 21]", 'genre'] = "International"
result.loc[result['all'] == "[2, 10, 171, 12, 17]", 'genre'] = "Folk"
result.loc[result['all'] == "[49, 18, 1235, 17]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[33, 362, 10, 17]", 'genre'] = "Pop"
result.loc[result['all'] == "[442, 10, 5]", 'genre'] = "Classical"
result.loc[result['all'] == "[250, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[169, 9, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[10, 12, 31]", 'genre'] = "Pop"
result.loc[result['all'] == "[17, 362, 10, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[41, 2, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[2, 130, 10, 619, 12, 504]", 'genre'] = "International"
result.loc[result['all'] == "[2, 130, 4, 10, 619, 179, 504]", 'genre'] = "International"
result.loc[result['all'] == "[2, 130, 619, 12, 504, 31]", 'genre'] = "International"
result.loc[result['all'] == "[2, 130, 10, 619, 17, 504]", 'genre'] = "Folk"
result.loc[result['all'] == "[2, 130, 296, 10, 362, 619, 15, 504]", 'genre'] = "International"
result.loc[result['all'] == "[2, 130, 100, 10, 619, 21, 504]", 'genre'] = "International"
result.loc[result['all'] == "[2, 130, 10, 619, 79, 504]", 'genre'] = "International"
result.loc[result['all'] == "[2, 130, 296, 10, 619, 15, 468, 504]", 'genre'] = "International"
result.loc[result['all'] == "[504, 2, 468, 15]", 'genre'] = "International"
result.loc[result['all'] == "[66, 2, 12, 77]", 'genre'] = "International"
result.loc[result['all'] == "[49, 10, 17]", 'genre'] = "Pop"
result.loc[result['all'] == "[250, 4, 5, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[98, 4, 12]", 'genre'] = "Jazz"
result.loc[result['all'] == "[169, 9, 12, 404, 85, 25]", 'genre'] = "Rock"
result.loc[result['all'] == "[296, 491, 15, 468, 21, 693]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[15, 4, 21, 495]", 'genre'] = "Jazz"
result.loc[result['all'] == "[17, 322, 5]", 'genre'] = "Classical"
result.loc[result['all'] == "[66, 12, 17, 58, 94]", 'genre'] = "Folk"
result.loc[result['all'] == "[26, 12, 236, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[58, 3, 12, 45]", 'genre'] = "Blues"
result.loc[result['all'] == "[12, 15, 400, 182, 26]", 'genre'] = "Electronic"
result.loc[result['all'] == "[2, 100, 21, 15]", 'genre'] = "International"
result.loc[result['all'] == "[98, 322, 4, 5, 12, 14, 19]", 'genre'] = "Jazz"
result.loc[result['all'] == "[41, 250, 4, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[66, 12, 21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[15, 21, 286, 181]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[4, 15, 400, 21, 182, 286]", 'genre'] = "Jazz"
result.loc[result['all'] == "[15, 468, 21, 181, 542]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[66, 100, 12, 15, 21]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[10, 66, 58, 12]", 'genre'] = "Pop"
result.loc[result['all'] == "[33, 38, 359, 12, 47, 17]", 'genre'] = "Experimental"
result.loc[result['all'] == "[296, 21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[10, 14]", 'genre'] = "Pop"
result.loc[result['all'] == "[250, 5, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[18, 1235, 314, 12]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[296, 18, 1235, 15]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[296, 12, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[9, 66, 651, 12]", 'genre'] = "Rock"
result.loc[result['all'] == "[741, 130, 2, 21]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[2, 4, 92]", 'genre'] = "International"
result.loc[result['all'] == "[98, 4, 297, 12, 15]", 'genre'] = "Jazz"
result.loc[result['all'] == "[12, 13, 17, 404, 85, 311]", 'genre'] = "Folk"
result.loc[result['all'] == "[130, 2, 296, 10, 619, 12, 15]", 'genre'] = "International"
result.loc[result['all'] == "[169, 12, 9]", 'genre'] = "Rock"
result.loc[result['all'] == "[33, 38, 47, 17, 250]", 'genre'] = "Experimental"
result.loc[result['all'] == "[9, 651, 4, 493]", 'genre'] = "Jazz"
result.loc[result['all'] == "[322, 5, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[17, 2, 130]", 'genre'] = "Folk"
result.loc[result['all'] == "[8, 2, 102, 130]", 'genre'] = "International"
result.loc[result['all'] == "[21, 693, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[8, 17, 137, 9]", 'genre'] = "Old-Time / Historic"
result.loc[result['all'] == "[2, 92, 21]", 'genre'] = "International"
result.loc[result['all'] == "[12, 21]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[495, 19, 14, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[4, 14, 15, 495, 19]", 'genre'] = "Jazz"
result.loc[result['all'] == "[66, 10, 12, 89, 25]", 'genre'] = "Pop"
result.loc[result['all'] == "[21, 468, 693, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[184, 21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[4, 21, 542, 15]", 'genre'] = "Jazz"
result.loc[result['all'] == "[4, 811, 15, 21, 542]", 'genre'] = "Jazz"
result.loc[result['all'] == "[13, 4, 311, 15]", 'genre'] = "Jazz"
result.loc[result['all'] == "[811, 4, 21, 15]", 'genre'] = "Jazz"
result.loc[result['all'] == "[296, 236, 14, 15, 19]", 'genre'] = "Electronic"
result.loc[result['all'] == "[58, 3, 12, 14]", 'genre'] = "Blues"
result.loc[result['all'] == "[33, 10, 12, 17, 58]", 'genre'] = "Pop"
result.loc[result['all'] == "[12, 45, 15, 53, 90, 31]", 'genre'] = "Electronic"
result.loc[result['all'] == "[33, 12, 17, 89, 25]", 'genre'] = "Folk"
result.loc[result['all'] == "[10, 12, 66, 362]", 'genre'] = "Pop"
result.loc[result['all'] == "[6, 38, 8, 428, 16, 20, 763]", 'genre'] = "Experimental"
result.loc[result['all'] == "[66, 137, 10, 9, 12]", 'genre'] = "Pop"
result.loc[result['all'] == "[360, 38, 10, 6]", 'genre'] = "Experimental"
result.loc[result['all'] == "[6, 38, 15, 16, 17, 763]", 'genre'] = "Experimental"
result.loc[result['all'] == "[362, 10, 468, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[88, 10, 12, 66]", 'genre'] = "Pop"
result.loc[result['all'] == "[8, 5]", 'genre'] = "Classical"
result.loc[result['all'] == "[130, 2, 619, 12, 109, 25, 31]", 'genre'] = "International"
result.loc[result['all'] == "[2, 130, 10, 619, 46]", 'genre'] = "International"
result.loc[result['all'] == "[659, 5, 1235]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[4, 13, 311]", 'genre'] = "Jazz"
result.loc[result['all'] == "[97, 322, 4, 5, 18, 1235]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[41, 18, 1235, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[100, 12, 21, 31]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[66, 12, 286, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[504, 2, 18, 1235]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[18, 1235, 12, 66]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[2, 17, 18, 1235, 504]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[2, 296, 15, 18, 1235, 504]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[66, 2, 12, 18, 1235, 504]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[64, 296, 12, 15, 18, 1235, 25]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[185, 21, 542, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[504, 2, 79, 15]", 'genre'] = "International"
result.loc[result['all'] == "[100, 21, 286, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[41, 10, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[21, 286, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[184, 2, 79, 15]", 'genre'] = "International"
result.loc[result['all'] == "[17, 41, 38, 33]", 'genre'] = "Experimental"
result.loc[result['all'] == "[296, 19, 14, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[17, 4, 12]", 'genre'] = "Jazz"
result.loc[result['all'] == "[38, 41, 18, 1235, 250]", 'genre'] = "Experimental"
result.loc[result['all'] == "[74, 13, 4, 810]", 'genre'] = "Jazz"
result.loc[result['all'] == "[41, 21, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[2, 130, 4, 524, 118]", 'genre'] = "International"
result.loc[result['all'] == "[296, 362, 10, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[17, 137, 12, 9]", 'genre'] = "Folk"
result.loc[result['all'] == "[137, 10, 9, 14, 19]", 'genre'] = "Pop"
result.loc[result['all'] == "[9, 2, 17]", 'genre'] = "Folk"
result.loc[result['all'] == "[17, 359, 12, 33]", 'genre'] = "Folk"
result.loc[result['all'] == "[170, 10, 13]", 'genre'] = "Pop"
result.loc[result['all'] == "[137, 10, 9]", 'genre'] = "Pop"
result.loc[result['all'] == "[10, 3, 567]", 'genre'] = "Blues"
result.loc[result['all'] == "[170, 10, 13, 14]", 'genre'] = "Pop"
result.loc[result['all'] == "[17, 3, 4]", 'genre'] = "Blues"
result.loc[result['all'] == "[4, 12, 49, 17, 58]", 'genre'] = "Jazz"
result.loc[result['all'] == "[2, 6, 38, 360, 10, 77]", 'genre'] = "Experimental"
result.loc[result['all'] == "[296, 14, 495, 15, 19]", 'genre'] = "Electronic"
result.loc[result['all'] == "[10, 2, 77, 14]", 'genre'] = "International"
result.loc[result['all'] == "[322, 4, 5]", 'genre'] = "Jazz"
result.loc[result['all'] == "[286, 21, 182, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[17, 10, 13]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 130, 10, 619, 17]", 'genre'] = "Folk"
result.loc[result['all'] == "[137, 5, 9]", 'genre'] = "Classical"
result.loc[result['all'] == "[4, 74, 12, 15, 26]", 'genre'] = "Jazz"
result.loc[result['all'] == "[2, 130, 296, 10, 15]", 'genre'] = "International"
result.loc[result['all'] == "[66, 137, 9, 12, 17]", 'genre'] = "Folk"
result.loc[result['all'] == "[19, 4, 14]", 'genre'] = "Jazz"
result.loc[result['all'] == "[10, 58, 12, 359]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 15, 79, 468, 21]", 'genre'] = "International"
result.loc[result['all'] == "[456, 5, 38, 47]", 'genre'] = "Experimental"
result.loc[result['all'] == "[296, 66, 12, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[98, 26, 5, 12, 442]", 'genre'] = "Classical"
result.loc[result['all'] == "[297, 12, 109, 15, 240, 25]", 'genre'] = "Electronic"
result.loc[result['all'] == "[2, 4, 297, 15, 504]", 'genre'] = "International"
result.loc[result['all'] == "[70, 297, 12, 15, 31]", 'genre'] = "Electronic"
result.loc[result['all'] == "[26, 12, 38, 47]", 'genre'] = "Experimental"
result.loc[result['all'] == "[10, 359, 12, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[10, 2, 12, 77]", 'genre'] = "International"
result.loc[result['all'] == "[10, 362, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[21, 542, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[4, 12, 15]", 'genre'] = "Jazz"
result.loc[result['all'] == "[314, 12, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[25, 12, 15, 297]", 'genre'] = "Electronic"
result.loc[result['all'] == "[66, 3, 12]", 'genre'] = "Blues"
result.loc[result['all'] == "[17, 468, 15]", 'genre'] = "Folk"
result.loc[result['all'] == "[3, 12, 45, 53, 90]", 'genre'] = "Blues"
result.loc[result['all'] == "[514, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[2, 4, 79]", 'genre'] = "International"
result.loc[result['all'] == "[19, 21, 14, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[906, 3, 4]", 'genre'] = "Blues"
result.loc[result['all'] == "[25, 12, 17, 33]", 'genre'] = "Folk"
result.loc[result['all'] == "[33, 3, 17]", 'genre'] = "Blues"
result.loc[result['all'] == "[10, 21, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[17, 10, 3]", 'genre'] = "Blues"
result.loc[result['all'] == "[10, 12, 109, 89, 25]", 'genre'] = "Pop"
result.loc[result['all'] == "[12, 21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[10, 21]", 'genre'] = "Pop"
result.loc[result['all'] == "[130, 2, 137, 9, 524, 17]", 'genre'] = "Folk"
result.loc[result['all'] == "[33, 12, 17, 15]", 'genre'] = "Folk"
result.loc[result['all'] == "[539, 21, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[2, 12, 46, 17, 25]", 'genre'] = "Folk"
result.loc[result['all'] == "[456, 41, 10, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[17, 10, 137, 9]", 'genre'] = "Pop"
result.loc[result['all'] == "[74, 250, 4, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[18, 1235, 5]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[33, 10, 17]", 'genre'] = "Pop"
result.loc[result['all'] == "[17, 3, 12]", 'genre'] = "Blues"
result.loc[result['all'] == "[10, 19, 14]", 'genre'] = "Pop"
result.loc[result['all'] == "[10, 12, 111, 25, 58]", 'genre'] = "Pop"
result.loc[result['all'] == "[25, 10, 12, 111]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 10, 171, 12]", 'genre'] = "International"
result.loc[result['all'] == "[2, 171, 15]", 'genre'] = "International"
result.loc[result['all'] == "[66, 10, 12, 404, 85]", 'genre'] = "Pop"
result.loc[result['all'] == "[514, 38, 41, 18, 1235]", 'genre'] = "Experimental"
result.loc[result['all'] == "[25, 12, 71, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[17, 3, 14]", 'genre'] = "Blues"
result.loc[result['all'] == "[9, 12, 63]", 'genre'] = "Rock"
result.loc[result['all'] == "[10, 58, 12]", 'genre'] = "Pop"
result.loc[result['all'] == "[66, 38, 456, 10, 12]", 'genre'] = "Experimental"
result.loc[result['all'] == "[10, 12, 359]", 'genre'] = "Pop"
result.loc[result['all'] == "[322, 5, 15, 18, 1235]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[296, 21, 542, 15]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[58, 3, 12, 567]", 'genre'] = "Blues"
result.loc[result['all'] == "[66, 12, 15, 400, 182, 58]", 'genre'] = "Electronic"
result.loc[result['all'] == "[322, 4, 5, 38, 137, 9, 17, 250]", 'genre'] = "Experimental"
result.loc[result['all'] == "[41, 1235, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[66, 74, 4, 12]", 'genre'] = "Jazz"
result.loc[result['all'] == "[10, 267, 1235, 15]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[232, 2, 4]", 'genre'] = "International"
result.loc[result['all'] == "[66, 2, 12, 79, 602]", 'genre'] = "International"
result.loc[result['all'] == "[296, 12, 70, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[66, 12, 15, 314, 286]", 'genre'] = "Electronic"
result.loc[result['all'] == "[10, 12, 70, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[2, 71, 12, 13, 311, 88, 25]", 'genre'] = "International"
result.loc[result['all'] == "[314, 12, 70, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[17, 2, 172]", 'genre'] = "Folk"
result.loc[result['all'] == "[514, 495, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[359, 10, 362, 12, 58]", 'genre'] = "Pop"
result.loc[result['all'] == "[1235, 15]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[2, 66, 10, 12, 77]", 'genre'] = "International"
result.loc[result['all'] == "[12, 236, 70, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[17, 41, 38, 15]", 'genre'] = "Experimental"
result.loc[result['all'] == "[33, 101, 12, 109, 17, 25, 31]", 'genre'] = "Folk"
result.loc[result['all'] == "[88, 17, 12, 49]", 'genre'] = "Folk"
result.loc[result['all'] == "[810, 13, 4, 21]", 'genre'] = "Jazz"
result.loc[result['all'] == "[17, 250, 38, 49]", 'genre'] = "Experimental"
result.loc[result['all'] == "[66, 12, 495, 15, 58]", 'genre'] = "Electronic"
result.loc[result['all'] == "[8, 1235]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[58, 12, 66, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[25, 12, 9, 63]", 'genre'] = "Rock"
result.loc[result['all'] == "[2, 4, 137, 9, 179, 118]", 'genre'] = "International"
result.loc[result['all'] == "[33, 810, 12, 13, 495, 15, 17, 88]", 'genre'] = "Folk"
result.loc[result['all'] == "[10, 286, 15]", 'genre'] = "Pop"
result.loc[result['all'] == "[17, 267, 1235]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[33, 12, 17, 36]", 'genre'] = "Folk"
result.loc[result['all'] == "[17, 58, 12, 98]", 'genre'] = "Folk"
result.loc[result['all'] == "[456, 18, 1235, 38]", 'genre'] = "Experimental"
result.loc[result['all'] == "[18, 1235, 659, 5]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[18, 1235, 12]", 'genre'] = "Instrumental"
result.loc[result['all'] == "[10, 3, 4]", 'genre'] = "Blues"
result.loc[result['all'] == "[130, 2, 12, 17, 89, 25]", 'genre'] = "Folk"
result.loc[result['all'] == "[10, 11, 14, 15, 182]", 'genre'] = "Pop"
result.loc[result['all'] == "[12, 36, 15]", 'genre'] = "Electronic"
result.loc[result['all'] == "[15, 337, 21, 542, 286]", 'genre'] = "Hip-Hop"
result.loc[result['all'] == "[542, 21, 286, 15]", 'genre'] = "Hip-Hop"

In [42]:
result.genre.value_counts()


Rock                   3965
Electronic             2527
Pop                    1392
Hip-Hop                1324
Folk                   1272
International           628
Jazz                    441
Experimental            440
Old-Time / Historic     376
Classical               321
Instrumental            236
Blues                   207
Name: genre, dtype: int64

In [43]:
result=result.drop(columns='all')
result=result.drop(columns=["('track', 'genres')"])

In [44]:
result.to_csv('model.csv')